In [4]:
import sys
sys.path.append('../src')

from wikidata_dumpreader import WikidataDumpReader
import json

FILEPATH = '../data/Wikidata/latest-all.json.bz2'
BATCH_SIZE = 100
NUM_PROCESSES = 4

wikidata = WikidataDumpReader(FILEPATH, num_processes=NUM_PROCESSES, batch_size=BATCH_SIZE)

outputfile = '../data/Wikidata/test.json'
def append_to_json(item):
    with open(outputfile, 'a+') as file:
        file.write(json.dumps(item)+",\n")

async def run_processor():
    await wikidata.run(append_to_json, max_iterations=3, verbose=True)


with open(outputfile, 'w+') as file:
    file.write("[")

await run_processor()

with open(outputfile, 'a+') as file:
    # Remove the last comma and new line
    file.seek(0, 2)
    file_size = file.tell()
    file.seek(file_size - 2)
    file.truncate()

    file.write("]")

    1: 187 (avg 187) items/sec 	 Memory Usage: 1292.73 MB
    2: 270 (avg 228) items/sec 	 Memory Usage: 1321.23 MB
    3: 355 (avg 270) items/sec 	 Memory Usage: 1321.98 MB


In [9]:
isEnglish['all']

23917869

In [12]:
isEnglish['all_size']

1454466.90625

In [1]:
import sys
sys.path.append('../src')

from wikidata_dumpreader import WikidataDumpReader
import json

FILEPATH = '../data/Wikidata/latest-all.json.bz2'
BATCH_SIZE = 10000
NUM_PROCESSES = 8

wikidata = WikidataDumpReader(FILEPATH, num_processes=NUM_PROCESSES, batch_size=BATCH_SIZE)

Wikitypes = {}
isEnglish = {}
total = 0
def count_types(item):
    global total
    global Wikitypes
    global isEnglish
    
    if item is not None:
        Wikitypes[item['type']] = Wikitypes.get(item['type'], 0) +1
        isEnglish['labels'] = isEnglish.get('labels', 0) + (1 if 'en' in item['labels'] else 0)
        isEnglish['descriptions'] = isEnglish.get('descriptions', 0) + (1 if 'en' in item['descriptions'] else 0)
        isEnglish['both'] = isEnglish.get('both', 0) + (1 if ('en' in item['descriptions']) and ('en' in item['labels']) else 0)
        if 'sitelinks' in item:
            isEnglish['wikipediaLink'] = isEnglish.get('both', 0) + (1 if 'enwiki' in item['sitelinks'] else 0)
            isEnglish['all'] = isEnglish.get('both', 0) + (1 if ('en' in item['descriptions']) and ('en' in item['labels']) and ('enwiki' in item['sitelinks']) else 0)
            isEnglish['all_size'] = isEnglish.get('all_size', 0) + (sys.getsizeof(item)/(1024**3) if ('en' in item['descriptions']) and ('en' in item['labels']) and ('enwiki' in item['sitelinks']) else 0)
        total += 1

async def run_processor():
    await wikidata.run(count_types, max_iterations=None, verbose=True)

await run_processor()

    1: 1032 (avg 1032) items/sec 	 Memory Usage: 3285.41 MB
    2: 1553 (avg 1292) items/sec 	 Memory Usage: 3292.11 MB


In [ ]:
def process_file_in_zip(zip_file, file_name):
    with zipfile.ZipFile(zip_file) as z:
        with z.open(file_name) as f:
            # `f` is an iterable, yielding lines from the file
            with ThreadPoolExecutor() as executor:
                executor.map(process_line, f) 

In [5]:
data = json.load(open(outputfile, 'r+'))
data[1]['labels']['en']
data[1]['descriptions']['en']
data[1]['aliases']['en']

[{'language': 'en', 'value': 'joy'}, {'language': 'en', 'value': 'happy'}]

In [34]:
data[1]['sitelinks']['enwiki']

{'site': 'enwiki', 'title': 'Belgium', 'badges': []}

In [18]:
data[1]['claims']['P361'][0]

{'mainsnak': {'snaktype': 'value',
  'property': 'P361',
  'datavalue': {'value': {'entity-type': 'item',
    'numeric-id': 215669,
    'id': 'Q215669'},
   'type': 'wikibase-entityid'},
  'datatype': 'wikibase-item'},
 'type': 'statement',
 'id': 'Q31$2BBFC346-FD6A-4EB1-89CC-5ABB0D5987F8',
 'rank': 'normal'}

In [29]:
data[1]['claims']['P571'][0]['mainsnak']

{'snaktype': 'value',
 'property': 'P571',
 'datavalue': {'value': {'time': '+1830-10-04T00:00:00Z',
   'timezone': 0,
   'before': 0,
   'after': 0,
   'precision': 11,
   'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'},
  'type': 'time'},
 'datatype': 'time'}